In [ ]:
size = 128
cur_dir = 'azure_storage_latency_graph/no_tunneling'
to_dc = ['central', 'south-central', 'west1']
config, to_dcs = get_configurations(0)
from_dc = 'central'
storage_latency = []
file_format = 'azure_storage_latency_raw/{}/azure_storage_get_{}kb.txt_{}_to_{}.log'
for dc in to_dc:
    storage_latency.append(get_giza_latency_raw_dir_map(file_format.format(from_dc, size, from_dc, dc))['main'])
bar_graph_with_error(storage_latency, '', '', to_dc, 'Latency (ms)', '{}/{}-{}-{}'.format(cur_dir, config, 'put', '256kb'), 301)

from collections import defaultdict
lf = []
lt = []
with open ('azure_storage_latency_raw/no_tunneling/blob_ss_h.log', 'r') as f:
    for line in f:
        lf.append(line)
with open ('azure_storage_latency_raw/no_tunneling/blob_ss_t.log', 'r') as f:
    for line in f:
        lt.append(line)
        
print len(lf)
print len(lt)
lf.sort()
lt.sort()

from collections import defaultdict
d1 = defaultdict(list)
d2 = defaultdict(list)
for i in lf:
    d1[i.split()[4][:-1]].append(i)
for i in lt:
    d2[i.split()[2]].append(i)
print len(d1)
print len(d2)
print d1.keys()
print d2.keys()
a = [len(d1[key]) for key in d1.keys()]
b = [len(d2[key]) for key in d2.keys()]
print a
print b

storages = ['blob.by4prdstr03a.store.core.windows.net.http',
           'blob.dm5prdstr02a.store.core.windows.net.http',
           'blob.sn4prdstr03a.store.core.windows.net.http',
           'blob.cy4prdstr01a.store.core.windows.net.http',
           'blob.db4prdstr01a.store.core.windows.net.http']
l1 = []
l2 = []
for s in storages:   
    a = d1[s]
    b = d2[s]
    a = sorted(a, key=lambda line: line.split()[2][:-1])
    b = sorted(b, key=lambda line: line.split()[4])
    l1.extend(a)
    l2.extend(b)
print len(l1)
print len(l2)


from collections import defaultdict
lf = []
lt = []
with open ('azure_storage_latency_raw/no_tunneling/blob_ss_h.log', 'r') as f:
    for line in f:
        lf.append(line)
with open ('azure_storage_latency_raw/no_tunneling/blob_ss_t.log', 'r') as f:
    for line in f:
        lt.append(line)
lf.sort()
lt.sort() 
lf = l1
lt = l2
current_string = ""
max_counter = 0
min_counter = 0
storages = ['blob.by4prdstr03a.store.core.windows.net.http',
           'blob.dm5prdstr02a.store.core.windows.net.http',
           'blob.sn4prdstr03a.store.core.windows.net.http',
           'blob.cy4prdstr01a.store.core.windows.net.http',
           'blob.db4prdstr01a.store.core.windows.net.http']
s_map = {
    'blob.by4prdstr03a.store.core.windows.net.http': 'west1',
    'blob.dm5prdstr02a.store.core.windows.net.http': 'central',
    'blob.sn4prdstr03a.store.core.windows.net.http': 'south-central',
    'blob.cy4prdstr01a.store.core.windows.net.http': 'japan-east',
    'blob.db4prdstr01a.store.core.windows.net.http': 'north-europe'
}
d = defaultdict(list)
ed = defaultdict(list)
counter1 = 0
counter2 = 0
supercounter = 0
while counter1 < len(lf) and counter2 < len(lt):
    if lf[counter1].split()[2] != lt[counter2].split()[4][:-1]:
        if counter1+1 >= len(lf) or counter2+1 >= len(lt):
            break
        if lf[counter1+1].split()[2] == lt[counter2].split()[4][:-1]:
            counter1 += 1
        if lf[counter2+1].split()[4][:-1] == lf[counter1+1].split()[2]:
            counter2 += 1
        else:
            supercounter += 1
            counter1 += 1
            counter2 += 1
#             print counter1
#             print counter2
#             break
    else:
        lfs = lf[counter1].split()
        lfs[4] = lfs[4][:-1]
        lts = lt[counter2].split()
        before = 0
        after = 0
        if lfs[4] in storages and lts[2] in storages:
            before = datetime.strptime(lfs[0], "%H:%M:%S.%f")
            after = datetime.strptime(lts[0], "%H:%M:%S.%f")
            if current_string == "":
                current_string = lfs[4]
                d[current_string + str(max_counter)].append(after-before)
                ed[s_map[current_string] + str(max_counter)].append(after-before)
            else:
                if current_string != lfs[4]:
                    if min_counter == 4:
                        min_counter = 0
                        max_counter += 1
                    else:
                        min_counter += 1
                    current_string = lfs[4]
                    d[current_string + str(max_counter)].append(after-before)
                    ed[s_map[current_string] + str(max_counter)].append(after-before)
                else:
                    d[current_string + str(max_counter)].append(after-before)
                    ed[s_map[current_string] + str(max_counter)].append(after-before)
        else:
            if lfs[4] != lts[2] and lts[2][-1] != 's' and lfs[4][-1] != 's':
                print 'error: ' + lfs[4] + '  ' + lts[2]
        counter1 += 1
        counter2 += 1
print supercounter

transfer_latency = [np.mean(ed['central3']), np.mean(ed['south-central3']), np.mean(ed['west13'])]
size = 128
cur_dir = 'azure_storage_latency_graph/no_tunneling'
to_dc = ['central', 'south-central', 'west1']
config, to_dcs = get_configurations(0)
from_dc = 'central'
storage_latency = []
file_format = 'azure_storage_latency_raw/{}/azure_storage_get_{}kb.txt_{}_to_{}.log'
for dc in to_dc:
    storage_latency.append(get_giza_latency_raw_dir_map(file_format.format(from_dc, size, from_dc, dc))['main'])

azure_latency = []
for i in range(len(storage_latency)):
    azure_latency.append(np.mean(storage_latency) - transfer_latency[i])
bar_graph_with_error(storage_latency, azure_latency, ('Transfer', 'Azure'), to_dc, 'Latency (ms)', '{}/{}-{}-{}'.format(cur_dir, config, 'put', '256kb'), 401)

west1put = [x.microseconds for x in ed['west10']]
np.mean(west1put)
size = 128
cur_dir = 'azure_storage_latency_graph/no_tunneling'
to_dc = ['central', 'south-central', 'west1']
config, to_dcs = get_configurations(0)
from_dc = 'central'
storage_latency = []
file_format = 'azure_storage_latency_raw/{}/azure_storage_get_{}kb.txt_{}_to_{}.log'
for dc in to_dc:
    storage_latency.append(get_giza_latency_raw_dir_map(file_format.format(from_dc, size, from_dc, dc))['main'])